# Feature Columns
Tensorflow offers a Feature columns API, which is the way we can pre-process our incoming datasets to adopt the shape most appropiated for our model.


In [6]:
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers

In [11]:


dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
file_path = '../datasets/petfinder-mini/petfinder-mini.csv'

# create dataset
test_dataset = tf.data.experimental.make_csv_dataset(file_path, batch_size=5, 
label_name='Type')

for features, target in test_dataset.take(1):
    print(target)    

tf.Tensor([b'Cat' b'Cat' b'Cat' b'Cat' b'Dog'], shape=(5,), dtype=string)


In [25]:
# Now lets define a demo function that will create a feature_layer
# with the specified feature column.
example_batch = next(iter(test_dataset))
features, target_label = example_batch
print(type(features))
print(type(target_label))
    
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(features).numpy())

photo_count = feature_column.numeric_column('PhotoAmt')
demo(photo_count)


<class 'collections.OrderedDict'>
<class 'tensorflow.python.framework.ops.EagerTensor'>
[[4.]
 [7.]
 [4.]
 [4.]
 [6.]]


In [26]:
# Bucketized Columns
age = feature_column.numeric_column('Age')
age_buckets = feature_column.bucketized_column(age, boundaries=[1,3,5])
demo(age_buckets)

[[0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]
